In [48]:
!pip install swifter

In [49]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os
import math
import datetime
import swifter

from datetime import date
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [50]:
cus = pd.read_csv("cus_info.csv")
iem = pd.read_csv("iem_info_20210902.csv")
hist = pd.read_csv("stk_bnc_hist.csv")
train = pd.read_csv("stk_hld_train.csv")
test = pd.read_csv("stk_hld_test.csv")
kospi = pd.read_csv("KOSPI Historical Data.csv")
submission = pd.read_csv("sample_submission.csv")

In [51]:
def get_date_mar(date_str):
  date_split = date_str.replace("-", "")
  return (date(int(date_split[:4]), int(date_split[4:6]), int(date_split[6:])))

monthly = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

def get_date(date_str):
  date_split = date_str.split(' ')
  for idx, month in enumerate(monthly):
    if (date_split[0] == month):
      ret_month = idx + 1
  ret = date(int(date_split[2]), ret_month, int((date_split[1])[:-1]))
  return ret

def get_price(price_str):
  price_split = price_str.replace(",", "")
  return float(price_split)

In [52]:
kospi["Date"] = kospi.swifter.apply(lambda row: get_date(row["Date"]), axis = 1)
kospi["Price"] = kospi.swifter.apply(lambda row: get_price(row["Price"]), axis = 1)

#휴장일
close_d = [date(2016, 1, 1), date(2016, 2, 8), date(2016, 2, 9), date(2016, 2, 10), date(2016, 3, 1), date(2016, 4, 13),
          date(2016, 5, 5), date(2016, 5, 6), date(2016, 6, 6), date(2016, 8, 15), date(2016, 9, 14),
          date(2016, 9, 15),
          date(2016, 9, 16), date(2016, 10, 3), date(2016, 12, 30),
          date(2017, 1, 27), date(2017, 1, 30), date(2017, 3, 1), date(2017, 5, 1), date(2017, 5, 3), date(2017, 5, 5),
          date(2017, 5, 9), date(2017, 6, 6), date(2017, 8, 15), date(2017, 10, 2), date(2017, 10, 3),
          date(2017, 10, 4),
          date(2017, 10, 5), date(2017, 10, 6), date(2017, 10, 9), date(2017, 12, 25), date(2017, 12, 29),
          date(2018, 1, 1), date(2018, 2, 15), date(2018, 2, 16), date(2018, 3, 1), date(2018, 5, 1), date(2018, 5, 7),
          date(2018, 5, 22), date(2018, 6, 6), date(2018, 6, 13), date(2018, 8, 15), date(2018, 9, 24),
          date(2018, 9, 25),
          date(2018, 9, 26), date(2018, 10, 3), date(2018, 10, 9), date(2018, 12, 25), date(2018, 12, 31),
          date(2019, 1, 1), date(2019, 2, 4), date(2019, 2, 5), date(2019, 2, 6), date(2019, 3, 1), date(2019, 5, 1),
          date(2019, 5, 6), date(2019, 6, 6), date(2019, 8, 15), date(2019, 9, 12), date(2019, 9, 13),
          date(2019, 10, 3),
          date(2019, 10, 9), date(2019, 12, 25), date(2019, 12, 31),
          date(2020, 1, 1), date(2020, 1, 24), date(2020, 1, 27), date(2020, 4, 15), date(2020, 4, 30),
          date(2020, 5, 1),
          date(2020, 5, 5), date(2020, 8, 17), date(2020, 9, 30), date(2020, 10, 1), date(2020, 10, 2),
          date(2020, 10, 9),
          date(2020, 12, 25), date(2020, 12, 31),
          date(2021, 1, 1), date(2021, 2, 11), date(2021, 2, 12), date(2021, 3, 1), date(2021, 5, 5),
          date(2021, 5, 19)]

Pandas Apply:   0%|          | 0/1227 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1227 [00:00<?, ?it/s]

In [53]:
def is_session(d): # 휴장일이 아니었는지 판단한다.
    if d.weekday() > 4:
        return False
    if d in close_d:
        return False
    return True

def trading_period(x, y): # 해당 날짜 사이의 영업일을 계산한다.
    if is_session(x) == False or is_session(y) == False:
        return None
    delta = (y - x).days
    if delta < 0:
        return None
    delta = delta + 1 - ((delta + x.weekday() + 1) // 7) * 2
    for i in close_d:
        if (i - x).days > 0 and (y - i).days > 0:
            delta = delta - 1
    return delta

def transf(x): # date형식으로 값을 변환해준다.
    year = x // 10000
    month = (x % 10000) // 100
    day = x % 100
    return date(year, month, day)

def corr(hold, hist, delta): # hist_d 값을 반환한다.
    if hist > hold or hold == hist + delta:
      return None
    return hist

def concat_dt(date): # 날짜를 int형식으로 변환해준다.
  return int(date.strftime('%Y') + date.strftime('%m') + date.strftime('%d'))

def get_set_d(byn_dt, hold_d):
  cnt = -1
  while (True):
    if (is_session(byn_dt)):
      cnt += 1
    if (cnt >= hold_d) and (is_session(byn_dt)):
      break
    byn_dt += datetime.timedelta(days = 1)
  return concat_dt(byn_dt)

def get_kospi_diff(byn_dt, sell_dt): # 해당 날짜의 kospi 지수 차를 구한다.
  try:
    buy_ksp = (kospi.loc[kospi["Date"] == byn_dt, ["Price"]]).values[0]
    sel_ksp = (kospi.loc[kospi["Date"] == sell_dt, ["Price"]]).values[0]
    return (float(sel_ksp) - float(buy_ksp))
  except Exception as e:
    return (None)

In [54]:
train["hist_d"] = 0
set_ref_day = date(2019, 12, 30) #ref_d 설정
set_fin_day = date(2020, 12, 30)
delta = trading_period(set_ref_day, set_fin_day) - 1

### train에 hist_d, kospi_diff 피쳐 추가

In [55]:
train["hist_d"] = train.swifter.apply(lambda row: trading_period(transf(row["byn_dt"]), set_ref_day), axis = 1)
train["hist_d"] = train.swifter.apply(lambda row: corr(row["hold_d"], row["hist_d"], delta), axis = 1) 
train.dropna(subset=['hist_d'], how='all', inplace=True)

train.insert(3, "kospi_diff", None)
train["kospi_diff"] = train.swifter.apply(lambda row: get_kospi_diff(transf(row["byn_dt"]), transf(get_set_d(transf(row["byn_dt"]), row["hist_d"]))), axis = 1) # 30일일 후후 코스피지수차
train.dropna(subset=['kospi_diff'], how='all', inplace=True)

train.reset_index(drop =True, inplace = True)

Dask Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/681472 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/4 [00:00<?, ?it/s]

### test에 kospi_diff 추가

In [56]:
test.insert(3, "kospi_diff", None)
test["kospi_diff"] = test.swifter.apply(lambda row: get_kospi_diff(transf(row["byn_dt"]), date(2020, 12, 30)), axis = 1)
test.reset_index(drop =True, inplace = True)

Dask Apply:   0%|          | 0/4 [00:00<?, ?it/s]

### train, test 베이스라인 예시대로 적절히 데이터프레임 병합

In [57]:
train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train_data, iem, how = "left", on = ["iem_cd"])

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test_data, iem, how = "left", on = ["iem_cd"])

train_label = train_data["hold_d"]
train_data.drop(["hold_d"], axis = 1, inplace = True)

hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"])
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])]
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["act_id", "iem_cd", "byn_dt", "bse_dt"], axis = 1)
test_data = test_data.drop(["act_id", "iem_cd", "byn_dt", "submit_id", "hold_d", "bse_dt"], axis = 1)

L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])

train_data.dropna(subset=['hist_d', 'kospi_diff'], how='all', inplace=True)

train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [58]:
train_data.tail()

,kospi_diff,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
19273,81.57,45.0,2,2,99,6,2,2,2,3,16,1851,10,1,2,264.0,34584000.0,500.0,131000.0
19274,49.85,28.0,1,6,0,5,2,5,3,5,2,1501,10,2,99,20.0,65600.0,100.0,3280.0
19275,45.43,125.0,1,6,0,5,2,5,3,5,2,984,7,1,1,2.0,38100.0,500.0,19050.0
19276,34.25,34.0,1,6,0,5,2,5,3,5,2,1330,14,2,99,1.0,56700.0,5000.0,56700.0
19277,-28.54,5.0,1,4,4,2,2,3,4,3,8,648,3,3,99,3041.0,3405920.0,500.0,1120.0


In [59]:
test_data.tail()

,kospi_diff,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
70591,240.02,23,1,6,0,5,2,5,3,5,2,1369,2,1,1,1.0,198500.0,500.0,198500.0
70592,624.10,103,1,6,0,5,2,5,3,5,2,444,2,99,99,50.0,372000.0,5000.0,7440.0
70593,492.95,858,1,4,4,2,2,3,4,3,8,1280,14,3,99,75057.0,3677793.0,100.0,49.0
70594,116.65,11,1,4,4,2,2,3,4,3,8,2373,8,2,99,96.0,11424000.0,2500.0,119000.0
70595,66.61,4,1,4,4,2,2,3,4,3,8,2371,8,2,1,50.0,11400000.0,2500.0,228000.0


### 모델학습

In [60]:
models = []

folds = KFold(n_splits=20)
for train_idx, val_idx in folds.split(train_data):
      
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
      
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
      
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
      
    models.append(model)

Training until validation scores don't improve for 300 rounds.
[500]	valid_0's l2: 6022.21	valid_0's rmse: 77.6029
[1000]	valid_0's l2: 5941.99	valid_0's rmse: 77.0843
[1500]	valid_0's l2: 5840.13	valid_0's rmse: 76.4207
[2000]	valid_0's l2: 5814.53	valid_0's rmse: 76.2531
Did not meet early stopping. Best iteration is:
[1999]	valid_0's l2: 5813.34	valid_0's rmse: 76.2453
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's l2: 3448.15	valid_0's rmse: 58.7209
Early stopping, best iteration is:
[251]	valid_0's l2: 3321.14	valid_0's rmse: 57.6294
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's l2: 4105.51	valid_0's rmse: 64.0742
[1000]	valid_0's l2: 4092.34	valid_0's rmse: 63.9714
Early stopping, best iteration is:
[759]	valid_0's l2: 4082.12	valid_0's rmse: 63.8914
Training until validation scores don't improve for 300 rounds.
[500]	valid_0's l2: 4151.89	valid_0's rmse: 64.4352
Early stopping, best iteration is:
[382]	valid_0's l2

In [61]:
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)
print(predict)

[242.52841014 479.85303207 210.96493586 ... 894.7448502   37.15058067
  36.87610106]


In [62]:
maxi_d = test["hist_d"] +146
maxi_d.to_numpy()
pre = np.vstack([predict, maxi_d])
submission["hold_d"] = np.round(pre.min(axis = 0))

submission.to_csv("s.csv", index = False)